# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import torch
if torch.cuda.is_available():device='cuda'
else:device='cpu'
# import random
#import os
import numpy as np
# import math

In [ ]:
#| export
def foo(): pass

In [ ]:
#| export

def bar():
    
    return 'hi'

In [ ]:
## | hide
import nbdev; nbdev.nbdev_export()